In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found 

In [2]:
#### Cell to run

import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.quantization import quantize_dynamic, get_default_qat_qconfig, prepare_qat, convert
from transformers import AdamW
# from datasets import load_metric
from tqdm import tqdm
# from transformers import pipeline
# import evaluate
import shutil
import evaluate
# from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [3]:
#### Cell to run

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
# model.train()
# model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
# torch.quantization.prepare_qat(model, inplace=True)


In [ ]:
#### Cell to run

def preprocess_texts(texts, src_lang_code, tgt_lang_code):
    processed_src_texts = []
    processed_tgt_texts = []
    for src_text, tgt_text in texts:
        src_text = f"{src_text} {tokenizer.eos_token} {src_lang_code}"
        tgt_text = f"{tgt_lang_code} {tgt_text} {tokenizer.eos_token}"
        processed_src_texts.append(src_text)
        processed_tgt_texts.append(tgt_text)
    return processed_src_texts, processed_tgt_texts

In [ ]:
#### Cell to run

class TranslationDataset(Dataset):
    def __init__(self, texts, tokenizer, src_lang_code, tgt_lang_code):
        self.texts = texts
        self.tokenizer = tokenizer
        self.src_lang_code = src_lang_code
        self.tgt_lang_code = tgt_lang_code
        self.src_texts, self.tgt_texts = preprocess_texts(texts, src_lang_code, tgt_lang_code)

    def __len__(self):
        return len(self.src_texts)

    def __getitem__(self, idx):
        src_text = self.src_texts[idx]
        tgt_text = self.tgt_texts[idx]

        inputs = self.tokenizer(src_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        labels = self.tokenizer(tgt_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        labels = labels.input_ids.squeeze()

        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }


In [ ]:

train_dutch = pd.read_csv('/content/drive/MyDrive/Dutch_train.csv')
val_dutch = pd.read_csv('/content/drive/MyDrive/Dutch_validation.csv')
test_dutch = pd.read_csv('/content/drive/MyDrive/Dutch_test.csv')

train_german = pd.read_csv('/content/drive/MyDrive/German_train.csv')
val_german = pd.read_csv('/content/drive/MyDrive/German_validation.csv')
test_german = pd.read_csv('/content/drive/MyDrive/German_test.csv')

train_spanish = pd.read_csv('/content/drive/MyDrive/Spanish_train.csv')
val_spanish = pd.read_csv('/content/drive/MyDrive/Spanish_validation.csv')
test_spanish= pd.read_csv('/content/drive/MyDrive/Spanish_test.csv')


In [ ]:
#### Cell to run

train_dutch = pd.read_csv('Dutch_train.csv')
val_dutch = pd.read_csv('Dutch_validation.csv')
test_dutch = pd.read_csv('Dutch_test.csv')

train_german = pd.read_csv('German_train.csv')
val_german = pd.read_csv('German_validation.csv')
test_german = pd.read_csv('German_test.csv')

train_spanish = pd.read_csv('Spanish_train.csv')
val_spanish = pd.read_csv('Spanish_validation.csv')
test_spanish = pd.read_csv('Spanish_test.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#### Cell to run

train_dutch = train_dutch[['English','Dutch']]
val_dutch = val_dutch[['English','Dutch']]
test_dutch = test_dutch[['English','Dutch']]

train_german = train_german[['English','German']]
val_german = val_german[['English','German']]
test_german = test_german[['English','German']]

train_spanish = train_spanish[['English','Spanish']]
val_spanish = val_spanish[['English','Spanish']]
test_spanish = test_spanish[['English','Spanish']]

In [ ]:
#### Cell to run

train_dutch = train_dutch[0:3000]
train_german = train_german[0:3000]
train_spanish = train_spanish[0:3000]

val_dutch = val_dutch[0:300]
val_german = val_german[0:300]
val_spanish = val_spanish[0:300]

test_dutch = test_dutch[0:150]
test_german = test_german[0:150]
test_spanish = test_spanish[0:150]

In [ ]:
###### Gathering results of basic pre-trained model

Dutch_results = pd.DataFrame()
Dutch_results['source'] = test_dutch['English']
Dutch_results['target'] = test_dutch['Dutch']
Dutch_results['NLLB-basic version'] = None
Dutch_results['NLLB-fine tuned version'] = None

German_results = pd.DataFrame()
German_results['source'] = test_german['English']
German_results['target'] = test_german['German']
German_results['NLLB-basic version'] = None
German_results['NLLB-fine tuned version'] = None

Spanish_results = pd.DataFrame()
Spanish_results['source'] = test_spanish['English']
Spanish_results['target'] = test_spanish['Spanish']
Spanish_results['NLLB-basic version'] = None
Spanish_results['NLLB-fine tuned version'] = None

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [ ]:
model.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",tgt_lang="nld_Latn")
encoded_en_text = tokenizer(Dutch_results['source'].to_list(), return_tensors="pt",truncation = True,padding=True).to(device)
generated_tokens = model.generate(**encoded_en_text).to(device)
Dutch_results['NLLB-basic version']= tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",tgt_lang="deu_Latn")
encoded_en_text = tokenizer(German_results['source'].to_list(), return_tensors="pt",truncation = True,padding=True).to(device)
generated_tokens = model.generate(**encoded_en_text).to(device)
German_results['NLLB-basic version']= tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",tgt_lang="spa_Latn")
encoded_en_text = tokenizer(Spanish_results['source'].to_list(), return_tensors="pt",truncation = True,padding=True).to(device)
generated_tokens = model.generate(**encoded_en_text).to(device)
Spanish_results['NLLB-basic version']= tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
Dutch_results.to_csv('Dutch_results_NLLB.csv')
German_results.to_csv('German_results_NLLB.csv')
Spanish_results.to_csv('Spanish_results_NLLB.csv')

In [ ]:
#### Dutch Language

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = Dutch_results['NLLB-basic version'].tolist()
references = Dutch_results['target'].tolist()

bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.11923532917558244, 'precisions': [0.3447680157946693, 0.1468477703741671, 0.08128997867803837, 0.04911209766925638], 'brevity_penalty': 1.0, 'length_ratio': 1.0214267708595917, 'translation_length': 4052, 'reference_length': 3967}
ROUGE score: {'rouge1': 0.3249658798161815, 'rouge2': 0.1620034745340809, 'rougeL': 0.2868364601140547, 'rougeLsum': 0.2875721882405544}
METEOR score: {'meteor': 0.30316458026638327}


In [ ]:
#### German Language

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = German_results['NLLB-basic version'].tolist()
references = German_results['target'].tolist()

bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.058498434345139134, 'precisions': [0.19940623453735776, 0.06731757451181912, 0.038214858364510954, 0.022828507795100223], 'brevity_penalty': 1.0, 'length_ratio': 1.0230321437610732, 'translation_length': 4042, 'reference_length': 3951}
ROUGE score: {'rouge1': 0.15912224256259583, 'rouge2': 0.0776042034702651, 'rougeL': 0.14371015826802214, 'rougeLsum': 0.14265779066284423}
METEOR score: {'meteor': 0.17211556746780304}


In [ ]:
#### Spanish Language

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = Spanish_results['NLLB-basic version'].tolist()
references = Spanish_results['target'].tolist()

bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.009633260802600364, 'precisions': [0.14516896715849595, 0.013079960513326752, 0.004100461301896463, 0.0015987210231814548], 'brevity_penalty': 0.9120148551708231, 'length_ratio': 0.9156679015035956, 'translation_length': 4202, 'reference_length': 4589}
ROUGE score: {'rouge1': 0.06855229714516653, 'rouge2': 0.006511404685572101, 'rougeL': 0.06349618410768512, 'rougeLsum': 0.06297227058326038}
METEOR score: {'meteor': 0.07920747873599288}


In [ ]:
#### Cell to run

english_dutch_texts = list(zip(train_dutch['English'].tolist(), train_dutch['Dutch'].tolist()))  # For Dutch translation
english_spanish_texts = list(zip(train_spanish['English'].tolist(), train_spanish['Spanish'].tolist()))
english_german_texts = list(zip(train_german['English'].tolist(), train_german['German'].tolist()))

datasets = [
    TranslationDataset(english_dutch_texts, tokenizer, "eng_Latn", "nld_Latn"),
    TranslationDataset(english_spanish_texts, tokenizer, "eng_Latn", "spa_Latn"),
    TranslationDataset(english_german_texts, tokenizer, "eng_Latn", "deu_Latn")
]

train_combined_dataset = ConcatDataset(datasets)
train_dataloader = DataLoader(train_combined_dataset, batch_size=2, shuffle=True,num_workers=8)

english_dutch_texts = list(zip(val_dutch['English'].tolist(), val_dutch['Dutch'].tolist()))  # For Dutch translation
english_spanish_texts = list(zip(val_spanish['English'].tolist(), val_spanish['Spanish'].tolist()))
english_german_texts = list(zip(val_german['English'].tolist(), val_german['German'].tolist()))

datasets = [
    TranslationDataset(english_dutch_texts, tokenizer, "eng_Latn", "nld_Latn"),
    TranslationDataset(english_spanish_texts, tokenizer, "eng_Latn", "spa_Latn"),
    TranslationDataset(english_german_texts, tokenizer, "eng_Latn", "deu_Latn")
]

eval_combined_dataset = ConcatDataset(datasets)
eval_dataloader = DataLoader(eval_combined_dataset, batch_size=2, shuffle=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    push_to_hub=False,
    report_to='none',
    logging_steps=10,
    dataloader_num_workers=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_combined_dataset,
    eval_dataset=eval_combined_dataset,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss
1,1.185600,1.300652
2,1.129200,1.289453
3,1.272300,1.291167


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation 

TrainOutput(global_step=6750, training_loss=1.2664345470004612, metrics={'train_runtime': 7697.565, 'train_samples_per_second': 3.508, 'train_steps_per_second': 0.877, 'total_flos': 7313978032128000.0, 'train_loss': 1.2664345470004612, 'epoch': 3.0})

In [ ]:
torch.save(model.state_dict(), 'NLLB_model_weights.pth')
import shutil



shutil.copy('NLLB_model_weights.pth', '/content/drive/My Drive/NLLB_model_weights.pth')

'/content/drive/My Drive/NLLB_model_weights.pth'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model.load_state_dict(torch.load('/content/drive/MyDrive/NLLB_model_weights.pth',map_location=torch.device('cuda')), strict=False)

<All keys matched successfully>

In [8]:
Dutch_results = pd.read_csv('Dutch_results_NLLB.csv')
German_results = pd.read_csv('German_results_NLLB.csv')
Spanish_results = pd.read_csv('Spanish_results_NLLB.csv')

In [10]:
device = 'cuda'

In [12]:
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",tgt_lang="nld_Latn")
encoded_en_text = tokenizer(Dutch_results['source'].to_list(), return_tensors="pt",truncation = True,padding=True).to(device)
generated_tokens = model.generate(**encoded_en_text).to(device)
Dutch_results['NLLB-fine tuned version']= tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",tgt_lang="deu_Latn")
encoded_en_text = tokenizer(German_results['source'].to_list(), return_tensors="pt",truncation = True,padding=True).to(device)
generated_tokens = model.generate(**encoded_en_text).to(device)
German_results['NLLB-fine tuned version']= tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [14]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",tgt_lang="spa_Latn")
encoded_en_text = tokenizer(Spanish_results['source'].to_list(), return_tensors="pt",truncation = True,padding=True).to(device)
generated_tokens = model.generate(**encoded_en_text).to(device)
Spanish_results['NLLB-fine tuned version']= tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [15]:
Dutch_results.to_csv('Dutch_results_NLLB.csv')
German_results.to_csv('German_results_NLLB.csv')
Spanish_results.to_csv('Spanish_results_NLLB.csv')

In [ ]:
#### Dutch Language

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = Dutch_results['NLLB-fine tuned version'].tolist()
references = Dutch_results['target'].tolist()

bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.051721981406496836, 'precisions': [0.21654501216545013, 0.0595959595959596, 0.031233595800524934, 0.01775471182736957], 'brevity_penalty': 1.0, 'length_ratio': 1.0360473909755483, 'translation_length': 4110, 'reference_length': 3967}
ROUGE score: {'rouge1': 0.18298769611395904, 'rouge2': 0.07345066458151639, 'rougeL': 0.1663009008000689, 'rougeLsum': 0.16644767186825976}
METEOR score: {'meteor': 0.17653177365592929}


In [ ]:
#### German Language

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = German_results['NLLB-fine tuned version'].tolist()
references = German_results['target'].tolist()

bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.04079611191375749, 'precisions': [0.15976047904191618, 0.04596273291925466, 0.025548387096774195, 0.01476510067114094], 'brevity_penalty': 1.0, 'length_ratio': 1.0566945077195646, 'translation_length': 4175, 'reference_length': 3951}
ROUGE score: {'rouge1': 0.10214856330959027, 'rouge2': 0.048130609896521286, 'rougeL': 0.09512251969339172, 'rougeLsum': 0.09653380863349045}
METEOR score: {'meteor': 0.132275151580891}


In [ ]:
#### Spanish Language

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = Spanish_results['NLLB-fine tuned version'].tolist()
references = Spanish_results['target'].tolist()

bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.1791309254634164, 'precisions': [0.41796786603303915, 0.22019208245490748, 0.14153920854576355, 0.09219143576826197], 'brevity_penalty': 0.9622603391258494, 'length_ratio': 0.9629548921333624, 'translation_length': 4419, 'reference_length': 4589}
ROUGE score: {'rouge1': 0.2956975465294389, 'rouge2': 0.17368021211399126, 'rougeL': 0.26671660344133424, 'rougeLsum': 0.26652377523376036}
METEOR score: {'meteor': 0.286850724265868}
